In [1]:
#!pip install neptune-client
import os
import neptune

#NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
#os.environ['NEPTUNE_PROJECT']="intelliscope/Test-HYBRID-Imbalanced"
#neptune.init('intelliscope/Test-HYBRID-Imbalanced',NEPTUNE_API_TOKEN)


In [ ]:
model.save(f'{modelsave_dir}/{str(exp)}.h5')
neptune.log_artifact(f'{modelsave_dir}/{str(exp)}.h5')

In [3]:
from neptune.sessions import Session
NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/HYBRID-Imbalanced']
projects=project.get_experiments()

In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *

'Experiment(HYBIM-1).h5'

In [ ]:
def get_results(model, PARAMS):
    X_test,Y_test = instead_data_loaders(PARAMS,'test')

    X_test_pcg = X_test[:,:,:,0:3]/255.0
    X_test_ecg = X_test[:,:,:,3:6]/255.0

    y_pred = model.predict([X_test[:,:,:,0:3],X_test[:,:,:,3:6]], batch_size=X_test.shape[0], verbose=1)

    y_pred_bool = np.argmax(y_pred, axis=1)
    y_test=np.argmax(Y_test,axis=1)


def neptune_log_results(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)

In [7]:
for project in projects:
    model_name=str(project)+'.h5'
    attr=project.get_parameters()
    
    model=load_model(attr['modelsave_dir']+'/'+model_name)
    
    y_test, y_pred=get_results(model, attr)
    neptune_log_metrics(y_test, y_pred)
    



{'notebook_name': 'hybrid01_imbalanced.ipynb',
 'name': 'data_a + imbalanced => Dilated-3',
 'description': 'Dilated Conv at first two , batch size 20 ',
 'tags': "['imbalanced', 'CASS', 'Plan 1', 'c5.12xlarge', 'Dilated Conv']",
 'data_balanced': 'False',
 'pretrained': 'False',
 'modelload_pcg_dir': '/home/ubuntu/intelliscope/models/Experiment(PCG-7).h5',
 'modelload_ecg_dir': '/home/ubuntu/intelliscope/models/Experiment(ECG-1).h5',
 'is_features_normalized': 'False',
 'Dilation_Rate': 3.0,
 'augmentation': 0.0,
 'opt': '<tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7fa497d92c50>',
 'dropout': 0.2,
 'num_epochs': 10.0,
 'wavelet': 'comb1',
 'batch_size': 20.0,
 'validation_split': 0.1,
 'test_split': 0.2,
 'dataset': 'physionet2016a',
 'pcg/ecg': 'pcg+ecg',
 'pcg_dir': '/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/pcg/imbalanced',
 'ecg_dir': '/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/ecg/imbalanced',
 'modelsave_dir': '/home/ubuntu/in

In [ ]:





exp = neptune.create_experiment(name=PARAMS['name'],description=PARAMS['description'],params=PARAMS,upload_source_files=PARAMS['notebook_name'],tags=PARAMS['tags'],upload_stdout=True)

model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))